<div class="alert alert-info">
    
➡️ Make sure that you have read the **[rules for hand-in assignments](https://www.ida.liu.se/~TDDE16/exam.en.shtml#handins)** and the **[policy on cheating and plagiarism](https://www.ida.liu.se/~TDDE16/exam.en.shtml#cheating)** before starting with this lab.

➡️ Make sure you fill in any cells (and _only_ those cells) that say **`YOUR CODE HERE`** or **YOUR ANSWER HERE**, and do _not_ modify any of the other cells.

➡️ **Before you submit your lab, make sure everything runs as expected.** For this, _restart the kernel_ and _run all cells_ from top to bottom. In Jupyter Notebook version 7 or higher, you can do this via "Run$\rightarrow$Restart Kernel and Run All Cells..." in the menu (or the "⏩" button in the toolbar).

</div>

# L1: Information Retrieval

In this lab you will apply basic techniques from information retrieval to implement the core of a minimalistic search engine. The data for this lab consists of a collection of app descriptions scraped from the [Google Play Store](https://play.google.com/store/apps?hl=en). From this collection, your search engine should retrieve those apps whose descriptions best match a given query under the vector space model.

In [81]:
# Define some helper functions that are used in this notebook

from IPython.display import display, HTML

def success():
    display(HTML('<div class="alert alert-success"><strong>Solution appears correct!</strong></div>'))

## Data set

The app descriptions come in the form of a compressed [JSON](https://en.wikipedia.org/wiki/JSON) file. Start by loading this file into a [Pandas](https://pandas.pydata.org) [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/dsintro.html#dataframe).

In [82]:
import bz2
import numpy as np
import pandas as pd

with bz2.open('app-descriptions.json.bz2') as source:
    df = pd.read_json(source)

In Pandas, a DataFrame is a table with indexed rows and labelled columns of potentially different types. You can access data in a DataFrame in various ways, including by row and column. To give an example, the code in the next cell shows rows 200–204:

In [83]:
df[200:205]

,name,description
200,Brick Breaker Star: Space King,Introducing the best Brick Breaker game that e...
201,Brick Classic - Brick Game,Classic Brick Game!\n\nBrick Classic is a popu...
202,Bricks Breaker - Glow Balls,Bricks Breaker - Glow Balls is a addictive and...
203,Bricks Breaker Quest,How to play\n- The ball flies to wherever you ...
204,Brothers in Arms® 3,Fight brave soldiers from around the globe on ...


As you can see, there are two labelled columns: `name` (the name of the app) and `description` (a textual description). The code in the next cell shows how to access fields from the description column.

In [84]:
df['description'][200:205]

200    Introducing the best Brick Breaker game that e...
201    Classic Brick Game!\n\nBrick Classic is a popu...
202    Bricks Breaker - Glow Balls is a addictive and...
203    How to play\n- The ball flies to wherever you ...
204    Fight brave soldiers from around the globe on ...
Name: description, dtype: object

## Problem 1: Preprocessing

Your first task is to implement a preprocessor for your search engine. In the vector space model, *preprocessing* refers to any transformation applied to a text before vectorisation. Here you can restrict yourself to a simple type of preprocessing: tokenisation, stop word removal, and lemmatisation.

To implement your preprocessor, you can use [spaCy](https://spacy.io). Make sure to read the [Linguistic annotations](https://spacy.io/usage/spacy-101#annotations) section of the spaCy&nbsp;101; that section contains all the information you need for this problem (and more).

Implement your preprocessor by completing the skeleton code in the next cell, adding additional code as you deem necessary.

In [85]:
import spacy
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'textcat'])

def preprocess(text):
    """Preprocess the given text by tokenising it, removing any stop words, 
    replacing each remaining token with its lemma (base form), and discarding 
    all lemmas that contain non-alphabetical characters.

    Arguments:
      text (str): The text to preprocess.

    Returns:
      The list of remaining lemmas after preprocessing (represented as strings).
    """
    # YOUR CODE HERE
    text_no_stop =[token for token in nlp(text) if not token.is_stop and token.is_alpha]
    
    text_lemma = [token.lemma_ for token in text_no_stop]
    return text_lemma 

    raise NotImplementedError()

In [86]:
preprocess('Apple is looking at buying U.K. startup for $1 billion')

['Apple', 'look', 'buy', 'startup', 'billion']

### 🤞 Test your code

Test your implementation by running the following cell:

In [87]:
"""Check that the preprocessing returns the correct output for a number of test cases."""

assert (
    preprocess('Apple is looking at buying U.K. startup for $1 billion') ==
    ['Apple', 'look', 'buy', 'startup', 'billion']
)
assert (
    preprocess('"Love Story" is a country pop song written and sung by Taylor Swift.') ==
    ['Love', 'Story', 'country', 'pop', 'song', 'write', 'sing', 'Taylor', 'Swift']
)
success()

## Problem 2: Vectorising

Your next task is to vectorise the data – and more specifically, to map each app description to a tf–idf vector. For this you can use the [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) class from [scikit-learn](https://scikit-learn.org/stable/). Make sure to specify your preprocessor from the previous problem as the `tokenizer` &ndash; not the `preprocessor`! &ndash; for the vectoriser. (In scikit-learn terminology, the `preprocessor` handles string-level preprocessing.)

After running the following cell:
- `vectorizer` should contain the vectorizer fitted on `df['description']`
- `X` should contain the vectorized `df['description']`

In [88]:
vectorizer, X = ..., ...

# YOUR CODE HERE
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=preprocess)
X = vectorizer.fit_transform(df['description'])




c:\Users\yj313\anaconda3\envs\environment_new_for_textmining\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 🤞 Test your code

Test your implementation by running the following cell:

In [89]:
"""Check that the dimensions of X are as expected."""

print(f"The dimensions of X are: {X.shape}")
assert X.shape[0] == 1614
assert 21200 < X.shape[1] < 21500

success()

The dimensions of X are: (1614, 21357)


The dimensions of `X` should be around 1614$\times$21356; the number of rows should be _exactly_ 1614 , while the number of columns may differ from that given here depending on the version of spaCy and the version of the language model used, as well as the pre-processing.

## Problem 3: Retrieving

To complete the search engine, your last task is to write a function that returns the most relevant app descriptions for a given query. An easy way to solve this task is to use scikit-learn&rsquo;s [NearestNeighbors](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html) class. That class implements unsupervised nearest neighbours learning and allows you to easily find a predefined number of app descriptions whose vector representations are closest to the query vector.

First, instantiate and fit a class that returns the _ten (10)_ nearest neighbors:

In [90]:
"""Instantiate and fit a class that returns the 10 nearest neighbors."""

# YOUR CODE HERE
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(X)

NearestNeighbors(n_neighbors=10)

Second, implement a function that uses the fitted class to find the nearest neighbours for a given query string:

In [91]:
from scipy.sparse import csr_matrix

In [92]:
def search(query):
    """Find the nearest neighbors in `df` for a query string.

    Arguments:
      query (str): A query string.

    Returns:
      The 10 apps (with name and description) most similar (in terms of
      cosine similarity) to the given query as a Pandas DataFrame.
    """
    # YOUR CODE HERE
    query_vector = vectorizer.transform([query])
    return df.iloc[neigh.kneighbors(query_vector, 10, return_distance=False)[0].tolist(),:]

### 🤞 Test your code

Test your implementation by running the following cell, which will show the 10 best search results for the query _"dodge trains"_:

In [93]:
"""Check that searching for "dodge trains" returns a DataFrame with ten results,
   and that the top result is "Subway Surfers"."""

result = search('dodge trains')
display(result)
assert isinstance(result, pd.DataFrame), "Search results should be a Pandas DataFrame"
assert len(result) == 10, "Should return 10 search results"
assert result.iloc[0]["name"] == "Subway Surfers", "Top search result should be 'Subway Surfers'"
success()

,name,description
1301,Subway Surfers,DASH as fast as you can! \nDODGE the oncoming ...
1428,Train Conductor World,Master and manage the chaos of international r...
1394,Tiny Rails,All aboard for an adventure around the world!\...
1300,Subway Princess Runner,"Subway princess runner, Bus run, forest rush w..."
998,No Humanity - The Hardest Game,2M+ Downloads All Over The World!\n\n* IGN Nom...
1429,Train for Animals - BabyMagica free,"🚂 BabyMagica ""Train for Animals"" is a educatio..."
1168,Rush,Are you ready for a thrilling ride?\n\nRush th...
786,LEGO® DUPLO® Train,All aboard! Driving the colorful LEGO® DUPLO® ...
1465,Virus War - Space Shooting Game,Warning! Virus invasion! Destroy them with you...
192,Boxing Star,Go for the K.O.!\nMake Your Opponent See Stars...


The top hit in the list should be _Subway Surfers_.

## Problem 4: Finding terms with low/high idf

Recall that the inverse document frequency (idf) of a term is the lower, the more documents from a given collection the term appears in. To get a better understanding for this concept, your next task is to write code to find out which terms from the app descriptions have the lowest/highest idf.

Start by sorting the terms in _increasing_ order of idf, breaking ties by falling back on alphabetic order, and store the result in the variable `terms`.

In [94]:
terms = []
# YOUR CODE HERE
idf = vectorizer.idf_
df_1 = pd.DataFrame({'idf': idf, 'terms': vectorizer.get_feature_names_out(input_features=None)})
df_1_sorted = df_1.sort_values(by=['idf','terms'])
df_1_sorted.head()
terms = df_1_sorted['terms'].tolist()




The following cell prints the 10 terms with the lowest/highest idf, which you can use to check if your results appear correct:

In [95]:
"""Print first 10/last 10 terms."""

print(f"Terms with the lowest idf:\n{terms[:10]}\n")
print(f"Terms with the highest idf:\n{terms[-10:]}")

Terms with the lowest idf:
['game', 'play', 'feature', 'free', 'new', 'world', 'time', 'app', 'fun', 'use']

Terms with the highest idf:
['회원가입에', '회원을', '획득한', '효과', '효과음', 'ﬁnd', 'ﬁnger', 'ﬁnish', 'ﬁrst', 'ﬂye']


## Problem 5: Keyword extraction

We often want to extract salient keywords from a document. A simple method is to pick the $k$ terms with the highest tf–idf value. Your last task in this lab is to implement this method. More specifically, we ask you to implement a function `keywords` that extracts keywords from a text.

In [96]:
def keywords(text, n=10):
    """
    Arguments:
      text (str): The text from which to extract keywords.
      n (int): The number of keywords to extract. [default: 10]

    Returns:
      A list containing the `n` most salient keywords from `text`, as measured by
      their tf–idf value relative to the collection of app descriptions.
    """
    # YOUR CODE HERE
    tfidf_text = vectorizer.transform([text]).toarray().flatten()
    df_2 = pd.DataFrame({'terms': vectorizer.get_feature_names_out(input_features=None),'dfidf':tfidf_text})
    df_2_sorted = df_2.sort_values(by='dfidf')
    return df_2_sorted['terms'][-n:]
    raise NotImplementedError()



### 🤞 Test your code

Test your implementation by running the following cell:

In [97]:
"""Check that the most salient keywords from the description of 'Train Conductor World'
   overlap substantially with the expected list of keywords."""

out = keywords(df['description'][1428])
print(out)
assert len(out) == 10
assert len(
    set(out) & set(['train', 'railway', 'railroad', 'rail', 'chaos', 'crash', 'timetable', 'overcast', 'haul', 'tram'])
) >= 6, "Keywords for df['description'][1428] do not overlap substantially with the expected result"
success()

9149     locomotive
16060     timetable
11286      overcast
16253          tram
3340          crash
2529          chaos
12658          rail
12659      railroad
12660       railway
16244         train
Name: terms, dtype: object


The cell above prints the most salient keywords from the description of the app "Train Conductor World". The exact output may differ slightly depending on the strategy used to break ties, so the cell only checks if there is a sufficient overlap.

**Congratulations on finishing this lab! 👍**

<div class="alert alert-info">
    
➡️ Don't forget to **test that everything runs as expected** before you submit!

</div>